In [99]:
import json

import pandas as pd
from rdkit import Chem

ps_path = './dataset_close_1.json'
gua_path = '../../guacamol2.csv'

with open(ps_path, 'r') as f:
    ps_data = json.load(f)

gua_data = pd.read_csv(gua_path)
gua_shred = gua_data[['smiles', 'scaffold_smiles']]
print(ps_data.keys())
ps_shred = [a[0] for a in ps_data['absorption']]
ps_smiles = list(set(ps_shred))
print(len(ps_shred), len(ps_smiles))

dict_keys(['absorption', 'emission', 'hl_gap', 'soqy', 'flqy', 'phosphorescence_peak'])
23234 10547


In [118]:
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmilesFromSmiles, MurckoScaffoldSmiles
from rdkit.Chem.Draw import MolToImage

ps_smiles_scaf = {
    'smiles':[],
    'scaffold_smiles':[],
}
for s in ps_smiles:
    m = Chem.MolFromSmiles(s)
    if m:
        scaf = MurckoScaffoldSmiles(mol=m)
        ps_smiles_scaf['smiles'].append(s)
        ps_smiles_scaf['scaffold_smiles'].append(scaf)
ps_df = pd.DataFrame(ps_smiles_scaf)

[11:06:28] Explicit valence for atom # 1 B, 4, is greater than permitted
[11:06:28] Explicit valence for atom # 14 N, 4, is greater than permitted
[11:06:28] Explicit valence for atom # 5 N, 4, is greater than permitted
[11:06:28] Explicit valence for atom # 7 B, 4, is greater than permitted
[11:06:29] Explicit valence for atom # 7 B, 4, is greater than permitted
[11:06:29] Explicit valence for atom # 14 B, 4, is greater than permitted
[11:06:29] Explicit valence for atom # 8 B, 4, is greater than permitted
[11:06:29] Explicit valence for atom # 5 N, 4, is greater than permitted
[11:06:29] Explicit valence for atom # 54 N, 4, is greater than permitted
[11:06:29] Explicit valence for atom # 1 B, 4, is greater than permitted
[11:06:29] Explicit valence for atom # 8 B, 4, is greater than permitted


[11:06:29] Explicit valence for atom # 27 N, 4, is greater than permitted
[11:06:29] SMILES Parse Error: syntax error while parsing: CC#CC1=CC(SC(C=C(C#CC2=CC=C(C)S2)C=C3)=C3N4[R])=C4C=C1
[11:06:29] SMILES Parse Error: Failed parsing SMILES 'CC#CC1=CC(SC(C=C(C#CC2=CC=C(C)S2)C=C3)=C3N4[R])=C4C=C1' for input: 'CC#CC1=CC(SC(C=C(C#CC2=CC=C(C)S2)C=C3)=C3N4[R])=C4C=C1'
[11:06:29] Explicit valence for atom # 7 B, 4, is greater than permitted
[11:06:29] Explicit valence for atom # 1 B, 4, is greater than permitted
[11:06:29] Explicit valence for atom # 1 B, 4, is greater than permitted
[11:06:29] Explicit valence for atom # 1 B, 4, is greater than permitted
[11:06:29] Explicit valence for atom # 1 B, 4, is greater than permitted
[11:06:29] Explicit valence for atom # 32 B, 4, is greater than permitted
[11:06:29] SMILES Parse Error: syntax error while parsing: CC#CC1=CC(SC(C=C(C#CC2=CC=C(C3=CC=C(C)C=C3)C=C2)C=C4)=C4N5[R])=C5C=C1
[11:06:29] SMILES Parse Error: Failed parsing SMILES 'CC#CC1=CC(SC

In [130]:
# concat guacamol and ps
new_df = pd.concat([gua_shred]+[ps_df]*10, ignore_index=True)

In [131]:
# duplicate smiles for nan, save new df
print(new_df.isnull().sum())
isnull = new_df['scaffold_smiles'].isnull()
for i in range(len(new_df)):
    if isnull[i]:
        new_df['scaffold_smiles'][i] = new_df['smiles'][i]

print(new_df.isnull().sum())
new_df.to_csv('../../guacamolps.csv')

smiles                 0
scaffold_smiles    15710
dtype: int64
smiles             0
scaffold_smiles    0
dtype: int64


In [126]:
# dropna, save new df
print(new_df.isnull().sum())
new_df = new_df.dropna()
print(new_df.isnull().sum())
new_df.to_csv('../../guacamolps_dropna.csv')

smiles                 0
scaffold_smiles    15710
dtype: int64
smiles             0
scaffold_smiles    0
dtype: int64


In [67]:
# save new df
# new_df = new_df.dropna()
new_df.to_csv('../../guacamolps_dropna.csv')

In [181]:
# read new df and dump as GGM dataset
# with open('../../GGM/data/id_smiles.txt') as f:
#     gd = f.read()
# print(gd.split('\n')[0].split('\t'))

read_data = pd.read_csv('../../guacamolps.csv', index_col=0)
print(new_df.shape, read_data.shape)
print(read_data.isnull().sum())
read_data = read_data.dropna()
print(read_data.isnull().sum())
write_data = []
for i in read_data.index:
    id = str(i).rjust(7, '0')
    name = f'GUACAPS-{id}'
    try:
        write_data.append('\t'.join([name, read_data['smiles'][i].replace('\n', ''), read_data['scaffold_smiles'][i].replace('\n', '')]))
    except AttributeError:
        print(name, read_data['smiles'][i], read_data['scaffold_smiles'][i])
written = '\n'.join(write_data)
with open('../../GGM/data/guacaps_dupsmiles.txt', 'w+') as f:
    f.write(written)

(1691438, 2) (1691438, 2)
smiles               0
scaffold_smiles    410
dtype: int64
smiles             0
scaffold_smiles    0
dtype: int64


In [183]:
# read new df and dump as GGM dataset
# with open('../../GGM/data/id_smiles.txt') as f:
#     gd = f.read()
# print(gd.split('\n')[0].split('\t'))

read_data = pd.read_csv('../../guacamolps_dropna.csv', index_col=0)
print(new_df.shape, read_data.shape)
print(read_data.isnull().sum())
read_data = read_data.dropna()
print(read_data.isnull().sum())
write_data = []
for i in read_data.index:
    id = str(i).rjust(7, '0')
    name = f'GUACAPS-{id}'
    try:
        write_data.append('\t'.join([name, read_data['smiles'][i].replace('\n', ''), read_data['scaffold_smiles'][i].replace('\n', '')]))
    except AttributeError:
        print(name, read_data['smiles'][i], read_data['scaffold_smiles'][i])
written = '\n'.join(write_data)
with open('../../GGM/data/guacaps_dropna.txt', 'w+') as f:
    f.write(written)

(1691438, 2) (1675728, 2)
smiles               0
scaffold_smiles    410
dtype: int64
smiles             0
scaffold_smiles    0
dtype: int64
